### Import Libraries

In [1]:
import json
import pandas as pd
import http.client
import requests
import datetime

In [2]:
#connection = http.client.HTTPConnection('api.football-data.org')
url = 'https://api.football-data.org/v2/competitions/PL/matches'
headers = { 'X-Auth-Token': '*********' }
response = requests.get(url, headers = headers)
result = json.loads(response.text)

In [4]:
import plotly
plotly.tools.set_credentials_file(username='adoh',api_key='eOypg265y6vHWAFV7GnP')
#with plotly
#import matplotlib.pyplot 
import plotly.plotly as py
from plotly.grid_objs import Grid, Column
from plotly.tools import FigureFactory as ff
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

In [12]:
dictn = {}
teamname = {}
i = 0
j = 0
for res in result['matches']:   
    if res['status'] != "FINISHED" :
        continue
    
    if res['score']['winner'] == 'DRAW' :
        for x in range(0,2):
            dictn[i] = {}
            dictn[i]['id'] = res['id']
            dictn[i]['mDay'] = res['matchday']
            dictn[i]['status'] = 'DRAW'
            
            dictn[i]['teamName'] = res['homeTeam']['name'] if x % 2 ==0 else res['awayTeam']['name']
            
            if dictn[i]['teamName'] not in teamname:
                teamname[dictn[i]['teamName']] = 1
            else:
                teamname[dictn[i]['teamName']] += 1
            dictn[i]['score'] = teamname[dictn[i]['teamName']]
            
           
            utcDate = datetime.datetime.strptime(res['utcDate'][:10],'%Y-%m-%d')
            i += 1
           
    else:
        dictn[i] = {}
        dictn[i]['id'] = res['id']
        dictn[i]['mDay'] = res['matchday']
        dictn[i]['teamName'] = res['homeTeam']['name'] if res['score']['winner'] == 'HOME_TEAM' else res['awayTeam']['name']
        
        
        if dictn[i]['teamName'] not in teamname:
            teamname[dictn[i]['teamName']] = 3
        else:
            teamname[dictn[i]['teamName']] += 3
        
        
        dictn[i]['score'] = teamname[dictn[i]['teamName']] 
        
        dictn[i]['status'] = 'WIN'
        i +=1
    

In [13]:
df4 = pd.DataFrame.from_dict(dictn,orient='index')
df4

,id,mDay,teamName,score,status
0,233028,1,Manchester United FC,3,WIN
1,233026,1,Tottenham Hotspur FC,3,WIN
2,233024,1,Crystal Palace FC,3,WIN
3,233029,1,Chelsea FC,3,WIN
4,233031,1,Watford FC,3,WIN
5,233032,1,AFC Bournemouth,3,WIN
6,233023,1,Wolverhampton Wanderers FC,1,DRAW
7,233023,1,Everton FC,1,DRAW
8,233025,1,Liverpool FC,3,WIN
9,233030,1,Southampton FC,1,DRAW


In [14]:
data = []
Nteam = ['Arsenal FC','Liverpool FC','Manchester City FC','Manchester United FC']
for win in Nteam:
    trace = go.Scatter(
        x = df4['mDay'].loc[(df4['teamName']==win)].sort_values(ascending=True),
        y = df4['score'].loc[(df4['teamName']==win)],
        name=win
    )
    data.append(trace)
plotly.offline.iplot(data,filename='basic-line')